In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torchvision import transforms, utils
from torch.utils.data import TensorDataset, DataLoader
import time

from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
with open("data/processed/training_data.pickle", "rb") as f:
    training_data = pickle.load(f)

with open("data/processed/training_arm.pickle", "rb") as f:
    training_arm = pickle.load(f)
    
with open("data/processed/mean_trajectory.pickle", "rb") as f:
    mean_trajectory = pickle.load(f)

In [3]:
data = np.concatenate((training_data, training_arm), axis=1)
data.shape

(40000, 3298)

In [4]:
BATCH_SIZE = 50

X_train, X_test, y_train_arm, y_test_arm = train_test_split(
    data[:, :297], data[:, 297:],
    test_size=0.3, random_state=2022
)

y_train = y_train_arm[:, 0]
y_test = y_test_arm[:, 0]

arm_train = y_train_arm[:, 1:]
arm_test = y_test_arm[:, 1:]

train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(arm_train))
valid_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(arm_test))

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
X, y = next(iter(train_dataloader))
print(X.shape)
print(y.shape)

torch.Size([50, 297])
torch.Size([50, 3000])


In [6]:
class NeuralDecoder(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(input_size, 500),
            nn.ReLU(),
            nn.Linear(500, 1_000),
            nn.ReLU(),
            nn.Linear(1_000, 5_000),
            nn.ReLU(),
            nn.Linear(5_000, 10_000),
            nn.ReLU(),
            nn.Linear(10_000, 5_000),
            nn.ReLU(),
            nn.Linear(5_000, output_size)
        )

    def forward(self, x):
        out = self.main(x)
        return out


def trainer(model, criterion, optimizer, dataloader, epochs=20, verbose=True):
    """Simple training wrapper for PyTorch network."""

    for epoch in range(epochs):
        losses = 0
        for X, y in dataloader:
            optimizer.zero_grad()       # Clear gradients w.r.t. parameters
            y_hat = model(X.reshape(X.shape[0], -1))
            loss = criterion(y_hat, y)  # Calculate loss
            loss.backward()             # Getting gradients w.r.t. parameters
            optimizer.step()            # Update parameters
            losses += loss.item()       # Add loss for this batch to running total
            
        if verbose:
            print(f"epoch: {epoch + 1}, loss: {losses / len(dataloader):.4f}")

In [7]:
model = NeuralDecoder(input_size=297, output_size=3_000)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8)

trainer(model, criterion, optimizer, train_dataloader, epochs=0, verbose=True)